# load in files

In [12]:
import json
import numpy as np



In [13]:
def load_json(fn):
    result = None
    with open(fn, "r") as in_file:
        result = json.load(in_file)
    return result



In [14]:
cards = load_json("./csvjson.json")


In [15]:
tags = load_json("./tags.json")


# load in graph


In [16]:
from py2neo import data, Graph, NodeMatcher, Node, Relationship, RelationshipMatcher


In [17]:
g = Graph(
    "neo4j+s://9262ab51.databases.neo4j.io:7687",
    auth=("neo4j", "FDpfCrT4h1SpR4CYcbAyM1-7CggziAnuqagatx5wL-o")
)


# create card nodes

In [18]:
cs = []
for x in cards:
    keyl = x['Key Learning']
    risk = x['Risks']
    asmp = x['Assumptions']
    goal = x['TNC 2030 Goals']
    jtbd = x['Conservation Job-to-be-done']
    tech = x['Tech Category']
    ttag = x['Tech Tags']
    orgs = x['Organization / BU']
    site = x['Project Site']
    desc = x['How we learnt this']
    coll = x['External Collaborators']
    ques = x['Open Questions']
    cont = x['Contacts']
    link = x['Links']
    tvsc = x['Tech vs Cons. Learning Type']
    cs.append([keyl, risk, asmp, goal, jtbd, tech, ttag, orgs, site, desc, coll, ques, cont, link, tvsc])
    
cypher_q9 = """
            create (t:Cards {key_learnings: $t_keyl, risks: $t_risk, assumptions: $t_asmp,
            goals: $t_goal, job_to_be_done: $t_jtbd, tech_category: $t_tech, tech_tags: $t_ttag,
            organizations_bus: $t_orgs, project_sites: $t_sites, how_we_learnt_this: $t_desc,
            external_collaborators: $t_coll, open_questions: $t_ques, contacts: $t_cont,
            links: $t_link, tech_vs_cons: $t_tvsc}) return t
            """

count = 0
for j in cs:
    result =  g.run(cypher_q9, t_keyl=j[0], t_risk=j[1], t_asmp=j[2], t_goal=j[3],
                   t_jtbd=j[4], t_tech=j[5], t_ttag=j[6], t_orgs=j[7], t_sites=j[8],
                   t_desc=j[9], t_coll=j[10], t_ques=j[11], t_cont=j[12], t_link=j[13], t_tvsc=j[14])
    count = count + 1

    

# 2030 goals tags and relationships

In [19]:
import pyparsing as pp

ts = []
for x in tags:
    tn = x['tagName']
    if tn == '2030Tags':
        for t in x['tag']:
            ts.append([tn,t])  

cypher_q1 = "create (t:Goals {tag: $t_tag, tagName: '2030 Goal'}) return t"
            
count = 0
for j in ts:
    tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
    for k in tt_arr:
        k_str = k.replace('"', '')
        result = g.run(cypher_q1, t_tag=k_str)
    count = count + 1



In [20]:


cypher_q10 = """
    match (g:Cards {goals: $t_goal}),
    (t:Goals {tag: $t_tag})
    merge (g)-[:tagged_goal]->(t)
"""


for r in cs:
    count = 0
    for p in ts:
        tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
        for q in tt_arr:
            q_str = q.replace('"', '')
            if q_str in r[3]:
                result = g.run(cypher_q10, t_goal=r[3], t_tag=q_str)
        count = count + 1


# triple a goals tags

In [21]:
ts = []
for x in tags:
    tn = x['tagName']
    if tn == 'projectGoals':
        for t in x['tag']:
            ts.append([tn,t])  

cypher_q2 = "create (t:ProjectGoals {tag: $t_tag, tagName: 'Project Goal'}) return t"
            
count = 0
for j in ts:
    tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
    for k in tt_arr:
        k_str = k.replace('"', '')
        result = g.run(cypher_q2, t_tag=k_str)
    count = count + 1



# conservation vs tech tags and relationships

In [22]:
ts = []
for x in tags:
    tn = x['tagName']
    if tn == 'techVsCons':
        for t in x['tag']:
            ts.append([tn,t])  

cypher_q16 = "create (t:TechVsCons {tag: $t_tag, tagName: 'Technical vs. Conservation'}) return t"
            
count = 0
for j in ts:
    tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
    for k in tt_arr:
        k_str = k.replace('"', '')
        result = g.run(cypher_q16, t_tag=k_str)
    count = count + 1




In [23]:
cypher_q17 = """
    match (g:Cards {tech_vs_cons: $t_tvsc}),
    (t:TechVsCons {tag: $t_tag})
    merge (g)-[:tagged_collabs]->(t)
"""

for r in cs:
    count = 0
    for p in ts:
        tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
        for q in tt_arr:
            q_str = q.replace('"', '')
            if q_str in r[10]:
                result = g.run(cypher_q17, t_tvsc=r[14], t_tag=q_str)
        count = count + 1

# conservation project to be done goals tags and relationships

In [24]:
ts = []
for x in tags:
    tn = x['tagName']
    if tn == 'projectTags':
        for t in x['tag']:
            ts.append([tn,t])  

cypher_q3 = "create (t:Project {tag: $t_tag, tagName: 'Project Goal'}) return t"
            
count = 0
for j in ts:
    tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
    for k in tt_arr:
        k_str = k.replace('"', '')
        result = g.run(cypher_q3, t_tag=k_str)
    count = count + 1



In [25]:
cypher_q11 = """
    match (g:Cards {job_to_be_done: $t_jtbd}),
    (t:Project {tag: $t_tag})
    merge (g)-[:tagged_project]->(t)
"""


for r in cs:
    count = 0
    for p in ts:
        tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
        for q in tt_arr:
            q_str = q.replace('"', '')
            if q_str in r[4]:
                result = g.run(cypher_q11, t_jtbd=r[4], t_tag=q_str)
        count = count + 1

# tech category tags and relationships

In [26]:
ts = []
for x in tags:
    tn = x['tagName']
    if tn == 'techCatTags':
        for t in x['tag']:
            ts.append([tn,t])  

cypher_q4 = "create (t:Tech {tag: $t_tag, tagName: 'Tech Categories'}) return t"
            
count = 0
for j in ts:
    tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
    for k in tt_arr:
        k_str = k.replace('"', '')
        result = g.run(cypher_q4, t_tag=k_str)
    count = count + 1
    


In [27]:
cypher_q11 = """
    match (g:Cards {tech_category: $t_tech}),
    (t:Tech {tag: $t_tag})
    merge (g)-[:tagged_tech]->(t)
"""


for r in cs:
    count = 0
    for p in ts:
        tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
        for q in tt_arr:
            q_str = q.replace('"', '')
            if q_str in r[5]:
                result = g.run(cypher_q11, t_tech=r[5], t_tag=q_str)
        count = count + 1

# tech subcategories tag and relationships

In [28]:
ts = []
for x in tags:
    tn = x['tagName']
    if tn == 'techSubCatTags':
        for t in x['tag']:
            ts.append([tn,t])  

cypher_q5 = "create (t:SpecificTech {tag: $t_tag, tagName: 'Tech Subcategories'}) return t"
            
count = 0
for j in ts:
    tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
    for k in tt_arr:
        k_str = k.replace('"', '')
        result = g.run(cypher_q5, t_tag=k_str)
    count = count + 1




In [29]:

cypher_q12 = """
    match (g:Cards {tech_tags: $t_ttag}),
    (t:SpecificTech {tag: $t_tag})
    merge (g)-[:tagged_spec_tech]->(t)
"""


for r in cs:
    count = 0
    for p in ts:
        tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
        for q in tt_arr:
            q_str = q.replace('"', '')
            if q_str in r[6]:
                result = g.run(cypher_q12, t_ttag=r[6], t_tag=q_str)
        count = count + 1

# BU tags and relationships

In [30]:
ts = []
for x in tags:
    tn = x['tagName']
    if tn == 'orgs/BUs':
        for t in x['tag']:
            ts.append([tn,t])  

cypher_q6 = "create (t:Organizations {tag: $t_tag, tagName: 'Organizations / BUs (TNC)'}) return t"
            
count = 0
for j in ts:
    tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
    for k in tt_arr:
        k_str = k.replace('"', '')
        result = g.run(cypher_q6, t_tag=k_str)
    count = count + 1




In [31]:

cypher_q13 = """
    match (g:Cards {organizations_bus: $t_orgs}),
    (t:Organizations {tag: $t_tag})
    merge (g)-[:tagged_orgs]->(t)
"""


for r in cs:
    count = 0
    for p in ts:
        tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
        for q in tt_arr:
            q_str = q.replace('"', '')
            if q_str in r[7]:
                result = g.run(cypher_q13, t_orgs=r[7], t_tag=q_str)
        count = count + 1

# sites tags and relationships

In [32]:

cypher_q7 = "create (t:Sites {tag: $t_tag, tagName: 'Sites / Preserves'}) return t"
            
count = 0
for j in ts:
    tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
    for k in tt_arr:
        k_str = k.replace('"', '')
        result = g.run(cypher_q7, t_tag=k_str)
    count = count + 1





In [33]:

cypher_q14 = """
    match (g:Cards {project_sites: $t_site}),
    (t:Sites {tag: $t_tag})
    merge (g)-[:tagged_sites]->(t)
"""


for r in cs:
    count = 0
    for p in ts:
        tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
        for q in tt_arr:
            q_str = q.replace('"', '')
            if q_str in r[8]:
                result = g.run(cypher_q14, t_site=r[8], t_tag=q_str)
        count = count + 1

# collabs tags and relationships

In [34]:
ts = []
for x in tags:
    tn = x['tagName']
    if tn == 'collabs':
        for t in x['tag']:
            ts.append([tn,t])  

cypher_q8 = "create (t:Collaborations {tag: $t_tag, tagName: 'External Collaborators'}) return t"
            
count = 0
for j in ts:
    tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
    for k in tt_arr:
        k_str = k.replace('"', '')
        result = g.run(cypher_q8, t_tag=k_str)
    count = count + 1





In [35]:

cypher_q15 = """
    match (g:Cards {external_collaborators: $t_coll}),
    (t:Collaborations {tag: $t_tag})
    merge (g)-[:tagged_collabs]->(t)
"""

for r in cs:
    count = 0
    for p in ts:
        tt_arr = pp.common.comma_separated_list.parseString(ts[count][1]).asList()
        for q in tt_arr:
            q_str = q.replace('"', '')
            if q_str in r[10]:
                result = g.run(cypher_q15, t_coll=r[10], t_tag=q_str)
        count = count + 1